Subtask3

In [0]:
import os
import numpy as np
from fever_io import load_dataset_json
from math import *
import re

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
PATH = '/content/gdrive/My Drive/IRDM_CHFX2/'

In [0]:
def n_dict_subtask3(filepath):
    '''
    The input is the path of wiki-pages.
    The output is the dictionary which the key is the term and the value is the frequency of the term.
    The output is 98 MB so it is not including in the file.
    '''
    n_dict = {}
    files = os.listdir(filepath)
    for i in files:
        with open(os.path.join(filepath, i)) as fp:
            lines = fp.readlines()
            for line in lines:
                text = eval(line)['text'] ## extract data from the field of 'text'.
                words = text.split(' ')
                for w in words:
                    w = w.replace("-LRB-","").replace("-RRB-","").replace("-LSB-","").replace("-RSB-","").replace("--","")
                    w = re.sub("[,.。:_=+*&^%$#@!?()<>/`';|]", "", w) ## replace the noisy with space. 
                    if not w in n_dict:
                        n_dict[w] = 1
                    else:
                        n_dict[w] += 1 ## count the frequencies of every term.
    np.save(PATH + "n_dict_Subtask3.npy",n_dict)
    print ('save complete')

In [0]:
n_dict_subtask3(PATH + 'data/wiki-pages/wiki-pages/')

save complete


In [0]:
def Subtask3_0(claim_id):
    '''
    The input is the claim id.
    The output is the claim, the 5 most similar documents and the query-likelihood unigram language model value.
    The out putis save in the 'Q3_unigram.csv'.
    In the query likelihood unigram language model, I do some smoothing to improve the result.
    For the terms in claim which do not appear in the document, the probability is not 0 but the probability it appear in the wiki-pages or p = 0.01 for the term even not appear in wiki-pages.
    '''
    alpha = 0.5

    train_data = load_dataset_json(PATH + 'data/train.jsonl', instance_num=20)

    claim = None
    for d in train_data:
        if d['id'] == claim_id:
            claim = d['claim'][:-1]
            claim = re.sub("[,.。:_=+*&^%$#@!?()<>/`';|]", "", claim)
            claim = claim.split(' ')
            break
    print(d['id'])
    print(d['claim'])

    data = np.load(PATH + 'n_dict_Subtask3.npy', allow_pickle=True).item()
    C = sum(data.values())
    f = []

    f = []
    files = os.listdir(PATH + 'data/wiki-pages/wiki-pages/')
    for i in files:
        with open(os.path.join(PATH + 'data/wiki-pages/wiki-pages/', i)) as fp:
            lines = fp.readlines()
            for line in lines:
                text = eval(line)['text']
                tmp = 0
                for w in claim:
                    if w in text:
                        p = text.count(w) / len(text) ## calculate the probability for the terms appear in the document.
                    else:
                        if w in data:
                            p = alpha * data[w] / C ## calculate the probability for the terms not appear in the document by using the probability it appear in the wiki-pages.
                        else:
                            p = 0.001 ## the probability of the terms do not appear in wiki-pages is 0.001.
                    tmp += log(p) ## calculate the log(p) of the claim.
                f.append((eval(line)['id'], tmp))
    f.sort(key=lambda x:x[1], reverse=True)
    return f[:5]

In [0]:
index_list = [
75397,
150448,
214861,
156709,
129629,
33078,
6744,
226034,
40190,
76253]

for index in index_list:
    print(Subtask3_0(index))

75397
Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
[('New_Amsterdam_-LRB-TV_series-RRB-', -59.87846610283076), ('Ved_verdens_ende', -59.99347182222764), ('Nikolaj_Coster-Waldau', -60.228888352654025), ('A_Thousand_Times_Good_Night', -60.996370163158), ('The_Other_Woman_-LRB-2014_film-RRB-', -61.42922043408902)]
150448
Roman Atwood is a content creator.
[('Bedside_Press', -35.14365238305695), ('Appticles', -38.396324446678605), ('Only_Much_Louder', -40.41470114501408), ('Roman_Atwood', -40.46609391817583), ('Abominable_Pictures', -40.67484796108127)]
214861
History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, film, photography and graphic arts.
[('History_of_art', -131.91786544237357), ('Fine_art', -135.19189978689758), ('World_Festival_of_Black_Arts', -135.6596758232308), ('The_arts', -135.7330800444195), ('Tamil_culture', -135.9928999920403)]
156709
Adrienne Bailon is an accountant.
[('Empire_Girls-COLON-_Ju

In [0]:
def Subtask3_Laplace(claim_id):
    '''
    The input is the claim id.
    The output is the claim, the 5 most similar documents and the Laplace Smoothing query-likelihood unigram language model value.
    The output is save in the 'Q3_laplace.csv'.
    '''
    alpha = 0.5

    train_data = load_dataset_json(PATH + 'data/train.jsonl', instance_num=20)

    claim = None
    for d in train_data:
        if d['id'] == claim_id:
            claim = d['claim'][:-1]
            claim = re.sub("[,.。:_=+*&^%$#@!?()<>/`';|]", "", claim)
            claim = claim.split(' ')
            break
    print(d['id'])
    print(d['claim'])

    data = np.load(PATH + 'n_dict_Subtask3.npy', allow_pickle=True).item()
    C = sum(data.values())
    f = []

    files = os.listdir(PATH + 'data/wiki-pages/wiki-pages/')
    for i in files:
        with open(os.path.join(PATH + 'data/wiki-pages/wiki-pages/', i)) as fp:
            lines = fp.readlines()
            for line in lines:
                text = eval(line)['text'].split(' ')
                tmp = 0
                for w in claim:
                    if w in text:
                        p = (text.count(w) + 1) / (len(text) + 1) ## calculate the probability for the terms appear in the document.
                    else:
                        if w in data:
                            p = alpha * (data[w] + 1) / (C + len(data)) ## calculate the probability for the terms not appear in the document by using the Laplace Smoothing.
                        else:
                            p = 0.01 ## the probability of the terms do not appear in wiki-pages is 0.001.
                    tmp += log(p) ## calculate the log(p) of the claim.
                f.append((eval(line)['id'], tmp))
    f.sort(key=lambda x:x[1], reverse=True)
    return f[:5]

In [0]:
index_list = [
75397,
150448,
214861,
156709,
129629,
33078,
6744,
226034,
40190,
76253]

for index in index_list:
    print(Subtask3_Laplace(index))

75397
Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
[('New_Amsterdam_-LRB-TV_series-RRB-', -50.76973653500242), ('The_Other_Woman_-LRB-2014_film-RRB-', -51.621240726089745), ('Nikolaj_Coster-Waldau', -52.08599794571365), ('Nukaaka_Coster-Waldau', -54.646680568578155), ('Fox_45', -54.832812336588766)]
150448
Roman Atwood is a content creator.
[('Only_Much_Louder', -33.31019819610523), ('Tosyn_Bucknor', -34.57610882268118), ('Stacey_Roy', -34.94560210320524), ('Joel_Spolsky', -35.094524034078326), ('Video_copy_detection', -35.19021271308817)]
214861
History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, film, photography and graphic arts.
[('History_of_art', -94.18366411935254), ('The_arts', -111.3534968720023), ('Fine_art', -111.50080770914465), ('Narrative', -115.17659264548013), ('World_Festival_of_Black_Arts', -116.47043356077488)]
156709
Adrienne Bailon is an accountant.
[('Empire_Girls-COLON-_Julissa_and_Adr

In [0]:
def Subtask3_JM(claim_id):
    '''
    The input is the claim id.
    The output is the claim, the 5 most similar documents and the Jelinek-Mercer Smoothing query-likelihood unigram language model value.
    The output is save in the 'Q3_jelinek_mercer.csv'.
    '''
    alpha = 0.5

    train_data = load_dataset_json(PATH + 'data/train.jsonl', instance_num=20)

    claim = None
    for d in train_data:
        if d['id'] == claim_id:
            claim = d['claim'][:-1]
            claim = re.sub("[,.。:_=+*&^%$#@!?()<>/`';|]", "", claim)
            claim = claim.split(' ')
            break
    print(d['id'])
    print(d['claim'])

    data = np.load(PATH + 'n_dict_Subtask3.npy', allow_pickle=True).item()
    C = sum(data.values())
    f = []

    files = os.listdir(PATH + 'data/wiki-pages/wiki-pages/')
    for i in files:
        with open(os.path.join(PATH + 'data/wiki-pages/wiki-pages/', i)) as fp:
            lines = fp.readlines()
            for line in lines:
                text = eval(line)['text'].split(' ')
                tmp = 0
                for w in claim:
                    if w in data:
                        p = alpha * text.count(w) / len(text) + (1 - alpha) * data[w] / C ## calculate the probability for the terms appear in the document by using the Jelinek-Mercer Smoothing.
                    else:
                        p = alpha * text.count(w) / len(text) ## calculate the probability for the terms not appear in the document by using the Jelinek-Mercer Smoothing.
                    tmp += log(p) ## calculate the log(p) of the claim.
                f.append((eval(line)['id'], tmp))
    f.sort(key=lambda x:x[1], reverse=True)
    return f[:5]

In [0]:
index_list = [
75397,
150448,
214861,
156709,
129629,
33078,
6744,
226034,
40190,
76253]

for index in index_list:
    print(Subtask3_JM(index))

75397
Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
[('New_Amsterdam_-LRB-TV_series-RRB-', -54.59930393193907), ('Nikolaj_Coster-Waldau', -55.55275862991747), ('The_Other_Woman_-LRB-2014_film-RRB-', -55.59365030594862), ('Ved_verdens_ende', -57.924043931964306), ('Nukaaka_Coster-Waldau', -57.95257191302444)]
150448
Roman Atwood is a content creator.
[('Only_Much_Louder', -37.872819586379364), ('Joel_Spolsky', -38.757393741621456), ('Tosyn_Bucknor', -39.14533053039745), ('Brett_Atwood', -39.23180145498373), ('Bedside_Press', -39.38852167039723)]
214861
History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, film, photography and graphic arts.
[('History_of_art', -112.9840675462393), ('The_arts', -124.22444576561661), ('Fine_art', -124.51267874848115), ('World_Festival_of_Black_Arts', -128.11753745058834), ('Narrative', -128.6320557285994)]
156709
Adrienne Bailon is an accountant.
[('Empire_Girls-COLON-_Julissa_and

In [0]:
def Subtask3_Dirichlet(claim_id):
    '''
    The input is the claim id.
    The output is the claim, the 5 most similar documents and the Dirichlet Smoothing query-likelihood unigram language model value.
    The output is save in the 'Q3_dirichlet.csv'.
    '''
    train_data = load_dataset_json(PATH + 'data/train.jsonl', instance_num=20)

    claim = None
    for d in train_data:
        if d['id'] == claim_id:
            claim = d['claim'][:-1]
            claim = re.sub("[,.。:_=+*&^%$#@!?()<>/`';|]", "", claim)
            claim = claim.split(' ')
            break
    print(d['id'])
    print(d['claim'])

    data = np.load(PATH + 'n_dict_Subtask3.npy', allow_pickle=True).item()
    C = sum(data.values())
    N = 5396106
    u = C/N
    f = []

    files = os.listdir(PATH + 'data/wiki-pages/wiki-pages/')
    for i in files:
        with open(os.path.join(PATH + 'data/wiki-pages/wiki-pages/', i)) as fp:
            lines = fp.readlines()
            for line in lines:
                text = eval(line)['text'].split(' ')
                alpha = u / (len(text) + u)
                tmp = 0
                for w in claim:
                    if w in data:
                        p = (text.count(w) + u * data[w] / C) / (len(text) + u) ## calculate the probability for the terms appear in the document by using the Dirichlet Smoothing.
                    else:
                        p = (text.count(w)) / (len(text) + u) ## calculate the probability for the terms not appear in the document by using the Dirichlet Smoothing.
                    tmp += log(p) ## calculate the log(p) of the claim.
                f.append((eval(line)['id'], tmp))
    f.sort(key=lambda x:x[1], reverse=True)
    return f[:5]

In [0]:
index_list = [
75397,
150448,
214861,
156709,
129629,
33078,
6744,
226034,
40190,
76253]

for index in index_list:
    print(Subtask3_Dirichlet(index))

75397
Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
[('New_Amsterdam_-LRB-TV_series-RRB-', -54.5011749198862), ('The_Other_Woman_-LRB-2014_film-RRB-', -56.45194982259717), ('Nikolaj_Coster-Waldau', -56.45628697367302), ('Ved_verdens_ende', -56.88079383622745), ('A_Second_Chance_-LRB-2014_film-RRB-', -57.69900355637624)]
150448
Roman Atwood is a content creator.
[('Joel_Spolsky', -38.748769703194654), ('Brett_Atwood', -39.25917567382504), ('Bedside_Press', -39.4132918475387), ('Only_Much_Louder', -39.605766950307526), ('Quetzal_-LRB-disambiguation-RRB-', -39.819512856347814)]
214861
History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, film, photography and graphic arts.
[('History_of_art', -106.66915996279059), ('The_arts', -123.82921183500564), ('Fine_art', -123.84171801351535), ('World_Festival_of_Black_Arts', -128.38190642559897), ('Narrative', -128.52255495085024)]
156709
Adrienne Bailon is an accountant.
[